In [2]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")


In [12]:
from penzai import pz
import json

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector


In [4]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

RuntimeError: Unable to initialize backend 'tpu': ABORTED: The TPU is already in use by process with pid 1484045. Not attempting to load libtpu.so in this process. (set JAX_PLATFORMS='' to automatically choose an available backend)

In [ ]:
from transformers import AutoTokenizer
import jax
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [10]:
from sprint.task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

In [15]:
import json, numpy as np

with open("cleanup_results_gemma_2_algo.jsonl", "r") as f:
    cleanup_results = [json.loads(line) for line in f][18:]


# with open("cleanup_results_gemma_2_all.jsonl", "r") as f:
#     tmp = [json.loads(line) for line in f]
#     tmp = [x for x in tmp if not x["task"].startswith("algo")]

#     cleanup_results += tmp

In [21]:

layers = [14, 16, 18, 20, 22, 24]

In [22]:
from micrlhf.utils.load_sae import sae_encode, get_dm_res_sae

thresholds = {
    layer: get_dm_res_sae(layer, load_65k=True).get("threshold", 0) for layer in layers
}


In [23]:
task = "antonyms"
task_results = [result for result in cleanup_results if result["layer"] in layers and result["task"] == task]   

print(len(task_results))

6


In [24]:
with open("gemma_2_cleaning_compact_65k.jsonl", "w") as f:
    for r in task_results:
        task = r["task"]
        layer = r["layer"]
        
        weights = np.array(r["weights"])

        i = np.argwhere(weights > thresholds[layer]).flatten()
        w = weights[i]

        idx = np.argsort(w)[::-1]

        i = i[idx]
        w = w[idx]

        data = {
            "task": task,
            "weights": w.tolist(),
            "indices": i.tolist(),
            "tv loss": r["tv_loss"],
            "cleaning loss": r["loss"],
            "ito loss": r["ito_loss"],
            "layer": layer
        }

        print(data)
        f.write(json.dumps(data) + "\n")

{'task': 'antonyms', 'weights': [51.93465042114258, 30.026456832885742, 19.8310489654541, 8.384634017944336, 7.020708084106445, 5.786473751068115], 'indices': [46209, 65229, 64797, 50557, 29998, 16645], 'tv loss': 2.875, 'cleaning loss': 1.765625, 'ito loss': 2.375, 'layer': 14}
{'task': 'antonyms', 'weights': [58.69618606567383, 38.361087799072266, 37.61246109008789, 33.8771858215332], 'indices': [59184, 56016, 710, 38724], 'tv loss': 1.6015625, 'cleaning loss': 1.65625, 'ito loss': 1.875, 'layer': 16}
{'task': 'antonyms', 'weights': [42.359962463378906, 38.76250457763672, 33.361412048339844, 26.453126907348633, 21.615398406982422, 18.74950408935547, 7.838016033172607, 4.738954067230225], 'indices': [50800, 37029, 44111, 1874, 4239, 8259, 3923, 36201], 'tv loss': 2.40625, 'cleaning loss': 3.046875, 'ito loss': 2.984375, 'layer': 18}
{'task': 'antonyms', 'weights': [68.01600646972656, 47.43989562988281, 34.77153396606445, 22.7160587310791, 12.102900505065918], 'indices': [17969, 35629,